## Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gurobipy import *

## Read and preprocess data


In [3]:
import csv
from datetime import datetime
import os

def load_data(file_path):
    data_dict = dict()

    with open(file_path) as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['Numéro du groupe'] != '':
                heures, minutes, _ = map(int, row['TransitTime'].split(':')) 
                total_minutes = heures * 60 + minutes
                data_dict[int(float(row['Numéro du groupe']))] = (
                    int(float(row['Femmes'])),
                    int(float(row['Hommes'])),
                    int(float(row['WCHR'])),
                    total_minutes
                )

    return data_dict

# Load data for each file

file_names = ["5Nov.csv", "7Nov.csv", "22Oct.csv", "24Oct.csv", "23Oct.csv", "30Oct.csv", "DataSeating 2024.csv"]
data_directory = 'data'
# for eachy dictionary : dict[Numéro du groupe : (Femmes, Hommes, WCHR, TransitTime)]
Nov5, Nov7, Oct22, Oct24, Oct23, Oct30, Date0 = [load_data(os.path.join(data_directory, file)) for file in file_names]

Example of how we access data

In [4]:
Nov5[1][0] # Number of women in group 1 on November 5th

1

Transforming data to access each passenger's Id

First we solved the problem for November 5th, then we'll have to automate it

In [58]:
Passengers = {}
j = 1

for group_num, data in Nov5.items():
    weights = {'Femmes': 70, 'Hommes': 85, 'WCHR': 10000}  # Weight per passenger
    
    # Extracting data
    femmes, hommes, wchr, total_minutes = data
    
    # Enumerating passengers
    for _ in range(femmes + hommes + wchr):
        if femmes > 0:
            weight = weights['Femmes']  
            Passengers[j] = {'gender': 'Femmes', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            femmes -= 1
        elif hommes > 0:
            weight = weights['Hommes']
            Passengers[j] = {'gender': 'Hommes', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            hommes -= 1
        else:
            weight = weights['WCHR']
            Passengers[j] = {'gender': 'WCHR', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            wchr -= 1
        j += 1

print(Passengers)

{1: {'gender': 'Femmes', 'group': 1, 'weight': 70, 'connection_time': 0}, 2: {'gender': 'Hommes', 'group': 1, 'weight': 85, 'connection_time': 0}, 3: {'gender': 'Hommes', 'group': 1, 'weight': 85, 'connection_time': 0}, 4: {'gender': 'Hommes', 'group': 2, 'weight': 85, 'connection_time': 60}, 5: {'gender': 'Femmes', 'group': 3, 'weight': 70, 'connection_time': 165}, 6: {'gender': 'Hommes', 'group': 3, 'weight': 85, 'connection_time': 165}, 7: {'gender': 'Femmes', 'group': 4, 'weight': 70, 'connection_time': 0}, 8: {'gender': 'Hommes', 'group': 4, 'weight': 85, 'connection_time': 0}, 9: {'gender': 'WCHR', 'group': 4, 'weight': 10000, 'connection_time': 0}, 10: {'gender': 'Femmes', 'group': 5, 'weight': 70, 'connection_time': 0}, 11: {'gender': 'Femmes', 'group': 5, 'weight': 70, 'connection_time': 0}, 12: {'gender': 'Femmes', 'group': 5, 'weight': 70, 'connection_time': 0}, 13: {'gender': 'Femmes', 'group': 6, 'weight': 70, 'connection_time': 0}, 14: {'gender': 'Hommes', 'group': 6, 'we

In [60]:
Passengers[1] # Characteristic of passenger 1 of November 5


{'gender': 'Femmes', 'group': 1, 'weight': 70, 'connection_time': 0}

## Static model

In [61]:
ranks = 29 #number of rows in the aircraft
n = len(Passengers) #number of passengers - TO BE CHANGED
ns=7*ranks #number of seats

def modele_statique():
    m=Model('statique') # Model initialization
    
    # -- Adding variables  --
    # Sij : dict[(int, int) : Var] : Associate passenger i with seat j
    S = {(i,j) : m.addVar(vtype = GRB.BINARY, name = f'j{i}') for i in range(1, n+1) for j in range (1, ns+1)}

    

    # -- Adding constraints  --
    
    # Max. 1 seat per passenger
    for j in range (1, ns+1):
        m.addConstr(sum(S[(i,j)] for i in range (1, n+1)) <=1, name="PassengerMax")

    #Each passenger has one and only one seat
    for i in range (1, n+1):
        m.addConstr(sum(S[(i,j)] for j in range (1, ns+1)) == 1, name="SeatMax")

    #The central aisle is left free
    for i in range (1, n+1):
        for j in range (1, ns+1):
            if j%7==4 : 
                m.addConstr(S[(i, j)] == 0)

    #Barycenter

    
    x_g = LinExpr()
    y_g = LinExpr()

    total_weight = quicksum(Passengers[i]['weight'] for i in range(1, n + 1)).getValue()

    for i in range(1, n + 1):
        for j in range(1, ns + 1):
            weight_ij = Passengers[i]['weight'] / total_weight
            if (j % 7 == 0):
                x_g += (1 * S[i, j] * weight_ij)
                y_g += (1 * S[i, j] * weight_ij)
            else:
                x_g += ((j % 7) * S[i, j] * weight_ij)
                y_g += ((j % 21) * S[i, j] * weight_ij)
    
    

    m.addConstr(x_g >= 3)  
    m.addConstr(x_g <= 5)  
    m.addConstr(y_g >= 13)  
    m.addConstr(y_g <= 17)


    #Passagers handicapés occupent 4 places collées à l'allée centrale
    for passenger in Passengers:
        if Passengers[passenger]['gender']=='WCHR':
            for j in range(1, ns+1-8, 7):             #On ârcourt les sièges par rangée (de 7 en 7)
            
                x1, x2, x3, x4, x5, x6, x7 =j, j+1, j+2, j+3, j+4, j+5, j+6
                
                
                m.addConstr(S[(passenger, x1)] + S[(passenger, x3)] + S[(passenger, x6)] + S[(passenger, x7)] ==0 )   #On fixe un siège dans les rangs 2 ou 5
                m.addConstr(S[(passenger, x2)]+sum(S[(i, x2+1)] for i in Passengers) <= 1) #On condamne le siège donnant sur l'allée
                m.addConstr(S[(passenger, x2)]+sum(S[(i, x2+7)] for i in Passengers) <= 1) #On condamne les 2 sièges derrière
                m.addConstr(S[(passenger, x2)]+sum(S[(i, x2+8)] for i in Passengers) <= 1)
                m.addConstr(S[(passenger, x5)]+sum(S[(i, x5+1)] for i in Passengers) <= 1) #On condamne le siège donnant sur l'allée
                m.addConstr(S[(passenger, x5)]+sum(S[(i, x5+7)] for i in Passengers) <= 1) #On condamne les 2 sièges derrière
                m.addConstr(S[(passenger, x5)]+sum(S[(i, x5+8)] for i in Passengers) <= 1)
                
            m.addConstr(sum(S[(passenger, k)] for k in range (ns-6, ns+1)) == 0)  #Un passger handicapé ne peut pas être sur la dernière rangée


    return m, S

In [62]:
## Fonctions objectifs

m,S=modele_statique()


Fonctions objectifs

In [63]:

#Les passagers en transit sont placés à l'avant de l'avion

def obj_transit():
    T= {} 
    
    
    for passenger in Passengers:
        if Passengers[passenger]['connection_time'] >0:   #passager en transit
            T[passenger]=Passengers[passenger]['connection_time']

    #T=sorted(T.items(), key=lambda item:item[1], reverse=True)
    P=list(T.keys())  #retourne la liste des identifiants des passagers en transit
    

    f=0
    for k in range (len(P)):
        for j in range (1, ns+1):
            q=(j-1)//7
            f+= S[(P[k],j)] * (1/T[P[k]]) * q
            

    return f

m.setObjective(obj_transit(), GRB.MINIMIZE)

In [64]:
# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet

# -- Mise à jour du modèle  --
m.update() 

# -- Affichage en mode texte du PL --
display(m)

<gurobi.Model MIP instance statique: 3905 constrs, 19691 vars, Parameter changes: Username=(user-defined), OutputFlag=0>

In [65]:
# -- Résolution --
m.optimize()

# -- Affichage de la solution --
print("Les places dans l'avion sont les suivantes :", [(i,j) for i in range(1,n+1) for j in range (1,ns+1) if S[(i,j)].x])



Les places dans l'avion sont les suivantes : [(1, 34), (2, 154), (3, 139), (4, 7), (5, 24), (6, 28), (7, 203), (8, 168), (9, 187), (10, 63), (11, 190), (12, 175), (13, 185), (14, 106), (15, 17), (16, 12), (17, 50), (18, 3), (19, 1), (20, 13), (21, 5), (22, 84), (23, 113), (24, 104), (25, 138), (26, 117), (27, 135), (28, 85), (29, 87), (30, 182), (31, 82), (32, 6), (33, 42), (34, 75), (35, 153), (36, 164), (37, 36), (38, 125), (39, 169), (40, 196), (41, 105), (42, 148), (43, 177), (44, 37), (45, 161), (46, 73), (47, 118), (48, 79), (49, 122), (50, 8), (51, 9), (52, 19), (53, 159), (54, 91), (55, 126), (56, 48), (57, 162), (58, 114), (59, 176), (60, 44), (61, 64), (62, 163), (63, 27), (64, 119), (65, 62), (66, 16), (67, 14), (68, 56), (69, 202), (70, 2), (71, 10), (72, 112), (73, 54), (74, 71), (75, 100), (76, 57), (77, 173), (78, 133), (79, 22), (80, 127), (81, 120), (82, 155), (83, 166), (84, 78), (85, 15), (86, 21), (87, 20), (88, 70), (89, 33), (90, 145), (91, 110), (92, 97), (93, 58